<a href="https://colab.research.google.com/github/dajoeng/KDT_team4_FinalPJT/blob/main/01_%EC%9B%B9%ED%81%AC%EB%A1%A4%EB%A7%81/%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%9D%B8%EA%B8%B0%EA%B2%80%EC%83%89%EC%96%B4%EC%B6%94%EC%B6%9C_%EB%B0%8F_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 일별로 업데이트됨
> 날짜가 넘어가면 하루 딱 한번 실행하는 식으로

# 설치

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 61.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 41.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/u

In [54]:
# -*- coding: UTF-8 -*-
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
import numpy as np
import itertools
import multiprocessing
from selenium.webdriver.chrome.options import Options
from datetime import datetime

# 밀키트 전체 인기 검색어 가져오기

In [ ]:
#인기 검색어 내용 추출
def get_rank_list(driver):
    trend_list = []
    rank_texts = driver.find_elements(by=By.CSS_SELECTOR, value='div.rank_top1000_scroll > ul')
    for rank_text in rank_texts:
        rank_text = rank_text.text
        rank_text = rank_text.split('\n')
        trend_list.extend(rank_text[1::2])
    return trend_list

#인기 검색어 1-40위까지 추출
def get_1_to_40(driver, TIMEOUT):
    trend_1_to_40 = []
    rank_text = get_rank_list(driver) #(1-20)
    trend_1_to_40.extend(rank_text)

    next_btn = driver.find_element(By.CSS_SELECTOR, 'div.top1000_btn_area > div > a.btn_page_next') #(21-40)
    next_btn.click()
    time.sleep(TIMEOUT)
    rank_text = get_rank_list(driver)
    trend_1_to_40.extend(rank_text)
    return trend_1_to_40

#인기 검색어 1-200위까지 추출
def get_1_to_200(driver, TIMEOUT):
    trend_1_to_200 = []
    rank_text = get_rank_list(driver) #(1-20)
    trend_1_to_200.extend(rank_text)

    for cnt in range(9):
      next_btn = driver.find_element(By.CSS_SELECTOR, 'div.top1000_btn_area > div > a.btn_page_next') #(21-200)
      next_btn.click()
      time.sleep(TIMEOUT)
      rank_text = get_rank_list(driver)
      trend_1_to_200.extend(rank_text)

    return trend_1_to_200

In [ ]:
# #Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
# url='https://datalab.naver.com/shoppingInsight/sCategory.naver?cid=50014240'
# TIMEOUT=3

# try:
#   driver = webdriver.ChromeOptions()
#   driver.add_argument('--headless')        # Head-less 설정
#   driver.add_argument('--no-sandbox')
#   driver.add_argument('--disable-dev-shm-usage')
#   driver = webdriver.Chrome('chromedriver', options=driver)
#   driver.get(url)
#   time.sleep(TIMEOUT)
  
#   #전체 밀키트 인기검색어
#   all_trend_list = []
#   all_trend_list.append(get_1_to_200(driver, TIMEOUT))

#   #세부카테고리의 인기 검색어 : '찌개/국','면/파스타','구이','볶음/튀김','조림/찜'
#   for idx in range(2,7):
#       driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span').click() #select_btn
#       time.sleep(TIMEOUT)
#       driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li['+str(idx)+']/a').click() #option
#       time.sleep(1.5)
#       driver.find_element(By.CSS_SELECTOR, '#content > div.section_instie_area.space_top > div > div.section.insite_inquiry > div > a').click() #btn_submit
#       time.sleep(1.5)
#       all_trend_list.append(get_1_to_200(driver, 1.5))
        
# except Exception:
#     raise

# finally:
#     if driver is not None:
#         driver.quit()

# 멀티 프로세스

- https://codewnag.tistory.com/26
- https://docs.python.org/ko/3.7/library/multiprocessing.html

In [ ]:
#------------------------------------------------------------
def do_multi(code):
  print ( str(code)+"번 프로세스 실행")
  
  # Colab에선 웹브라우저 창이 뜨지 않으므로 별도 설정한다.
  chrome_options = Options()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver', options=chrome_options)

  # 접속
  driver.get(url)
  time.sleep(TIMEOUT)

  #밀키트 전체
  driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span').click() #대분류 select_btn
  time.sleep(TIMEOUT)
  driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[7]/a').click() #option
  time.sleep(TIMEOUT)
  driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/span').click() #중분류 select_btn
  time.sleep(TIMEOUT)
  driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[2]/ul/li[22]/a').click() #option
  time.sleep(TIMEOUT)

  if not code == 1:
    #세부카테고리의 인기 검색어 : '밥/죽, '찌개/국','면/파스타','구이','볶음/튀김','조림/찜','간식/디저트'
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/span').click() #소분류 select_btn
    time.sleep(TIMEOUT)

  if code == 2:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[1]/a').click() #option
  elif code == 3:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[2]/a').click() #option
  elif code == 4:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[3]/a').click() #option
  elif code == 5:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[4]/a').click() #option
  elif code == 6:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[5]/a').click() #option
  elif code == 7:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[6]/a').click() #option
  elif code == 8:
    driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[3]/ul/li[7]/a').click() #option
  else: 
    pass

  #한번에 적용버튼 클릭 및 트렌드 추출
  time.sleep(TIMEOUT)
  driver.find_element(By.CSS_SELECTOR, '#content > div.section_instie_area.space_top > div > div.section.insite_inquiry > div > a').click() #btn_submit
  time.sleep(TIMEOUT)

  all_trend_list = []
  all_trend_list.append(get_1_to_200(driver, TIMEOUT))

  return all_trend_list

#-----------------------------------------------------------
def main():
  code_list = [id for id in range(1,9)] #창 개수
  print ('--- start _multiprocessing')

  # cpu 갯수 확인
  cpu_count = multiprocessing.cpu_count()
  print ('--- cpu_count ', cpu_count)

  # cpu 수 결정
  pool = multiprocessing.Pool(6)

  # 실행 함수, 넘겨줄 파라미터
  all_trend_list = pool.map(do_multi, code_list)

  # 모든 프로세스 종료까지 기다림
  pool.close()
  pool.join()
  # multi end

  return all_trend_list

In [ ]:
#실행코드
if __name__ == '__main__':
  global TIMEOUT, url

  TIMEOUT=1
  url='https://datalab.naver.com/shoppingInsight/sCategory.naver?cid=50014240'
  all_trend_list = main()

--- start _multiprocessing
--- cpu_count  2
2번 프로세스 실행
1번 프로세스 실행3번 프로세스 실행4번 프로세스 실행

5번 프로세스 실행6번 프로세스 실행


7번 프로세스 실행
8번 프로세스 실행


## 데이터프레임으로 정리

In [ ]:
#데이터프레임으로 정리
detailed_category = ['전체','밥/죽','찌개/국','면/파스타','구이','볶음/튀김','조림/찜','간식/디저트']
trend_df = pd.DataFrame(list(itertools.chain(*all_trend_list))).T
trend_df.columns = detailed_category

%cd /content/drive/My Drive
year = datetime.today().year
month = datetime.today().month
day = datetime.today().day
trend_df.to_csv(str(year) + '-' + str(month) + '-' + str(day) + 'navertrend.csv')

trend_df.info()

/content/drive/My Drive
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   전체      200 non-null    object
 1   밥/죽     200 non-null    object
 2   찌개/국    200 non-null    object
 3   면/파스타   200 non-null    object
 4   구이      200 non-null    object
 5   볶음/튀김   200 non-null    object
 6   조림/찜    200 non-null    object
 7   간식/디저트  200 non-null    object
dtypes: object(8)
memory usage: 12.6+ KB


In [ ]:
trend_df

,전체,밥/죽,찌개/국,면/파스타,구이,볶음/튀김,조림/찜,간식/디저트
0,밀키트,참한전복,밀키트,밀키트,밀키트,밀키트,캠핑요리추천,떡볶이
1,떡볶이,리조또밀키트,낙곱새,동남아식밀키트탄탄면,스테이크밀키트,감바스밀키트,캠핑음식추천,밀키트
2,낙곱새,프레시지리조또,알탕,파스타밀키트,자미곱,감바스,밀키트,쌀떡볶이
3,캠핑밀키트,옹기전팥죽,캠핑밀키트,수제비밀키트,목살밀키트,애슐리밀키트,찜닭밀키트,국물떡볶이
4,알탕,빠에야밀키트,해신탕,공항칼국수,군고기밀키트,돈이곱창참숯초벌냄새없는곱창볶음순대볶음간단캠핑요리밀키트돼지곱창볶음600g,갈비찜오리지널맛,사과떡볶이
...,...,...,...,...,...,...,...,...
195,캠핑요리,부타동밀키트,조개탕밀키트,코델리아파스타,주왕산돼지갈비,닭갈비캠핑요리생생닭갈비캠핑닭갈비,전자렌지밀키트,밀키트밀키트진짜떡볶이밀키트수제밀키트
196,허챠밍밀키트,분말죽,마이셰프밀푀유나베,금산제면소밀키트,야심한닭,캠핑닭갈비닭갈비900g생생닭갈비,찜갈비밀키트,즉석밀키트수제밀키트
197,파파쿡,비비고소고기죽280,돼지두루치기,맛좀봐라,연타발,생생닭갈비캠핑닭갈비춘천생생닭갈비,프레시지,진짜떡볶이밀키트수제밀키트즉석밀키트
198,촬떠그떡볶이,비비고전복죽450,대한곱창밀키트,베트남쌀국수밀키트,랍스터밀키트,순대밀키트,홍수계찜닭밀키트,즉석밀키트떡볶이밀키트


# konlpy, Mecab 형태소 분석기 설치 스크립트 실행

## mecab의 폴더를 보면 크게 3개의 폴더로 이뤄져 있습니다.
- mecab-ko-dic : mecab의 단어 사전
- tools : 단어 사전에 단어를 추가시킨 후 컴파일 및 적용을 시켜주는 프로그램들 존재
- user-dic : 사용자가 추가적으로 추가한 단어에 대한 단어 사전

## 단어 사전에 단어를 추가하는 프로세스는 크게 다음과 같이 진행이 됩니다.
1. user-dic의 csv 파일에 형식에 맞게 추가하고 싶은 단어를 추가
2. tools의 add-userdic-win.ps1라는 프로그램을 실행하여 컴파일 진행하여 추가 사항을 적용
3. 2번의 과정에 의해서 user-dic에서 설정한 csv파일이 mecab-ko-dic에 새로 csv파일이 만들어져서 단어 사전에 해당 단어가 적용됨

## mecab-ko-dic 품사 태그 설명
- http://eunjeon.blogspot.com/2014/03/blog-post.html
- https://docs.google.com/spreadsheets/d/1-9blXKjtjeKZqsf4NzHeYJCrr49-nXeRF6D80udfcwY/edit#gid=589544265
- 표층형 / 좌문맥ID / 우문맥ID / 비용(우선순위) /	품사 / 태그 /	의미부류 / 종성 유무 / 	읽기 / 타입 /	첫번째 품사 /	마지막 품사 /	원형 / 인덱스 표현

In [5]:
# konlpy, Mecab 형태소 분석기 설치 스크립트 실행
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash ./install_mecab-ko_on_colab190912.sh
%cd /content/mecab-ko-dic-2.1.1-20180720/
!pip install jamo

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 115, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 115 (delta 11), reused 10 (delta 3), pack-reused 91
Receiving objects: 100% (115/115), 1.27 MiB | 23.23 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.6/465.6 KB 41.8 MB/s eta 0:00:00
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2023-01-04 00:31:06--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to 

# 사용자 단어 사전 추가

In [39]:
from jamo import h2j, j2hcj
def get_nnpFile(word_list):
  #형식에 맞춰서 nnp 사전에 추가
  for word in word_list:
    sample_text_list = list(word)
    last_word = sample_text_list[-1]
    last_word_jamo_list = list(j2hcj(h2j(last_word)))
    last_jamo = last_word_jamo_list[-1]

    ## 마지막 글자가 받침을 가지느냐 안가지느냐로 T는 True, F는 False.
    jongsung_TF = "T"
    if last_jamo in ['ㅏ', 'ㅑ', 'ㅓ', 'ㅕ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', 'ㅘ', 'ㅚ', 'ㅙ', 'ㅝ', 'ㅞ', 'ㅢ', 'ㅐ,ㅔ', 'ㅟ', 'ㅖ', 'ㅒ']:
      jongsung_TF = "F"

    #표층형 / 좌문맥ID / 우문맥ID / 비용(우선순위) / 품사 / 태그 / 의미부류 / 종성 유무 / 읽기 / 타입 / 첫번째 품사 / 마지막 품사 / 원형 / 인덱스 표현
    line = '{},,,0,NNP,*,{},{},*,*,*,*,*\n'.format(word, jongsung_TF, word)

    file_data.append(line)

  return file_data

#--------------------------------------------------
%cd /content/mecab-ko-dic-2.1.1-20180720/
with open("./user-dic/nnp.csv", 'r', encoding='utf-8') as f:
  file_data = f.readlines()

#사용자 단어 정의 및 nnp파일에 추가
word_list = ['곱도리탕','소곱창','가니쉬','우족찜','밀키트','투움바','감바스','프레시지','피코크','파스타',
             '된장찌개','마라탕','해신탕','해물탕','낙곱새','낙새','오꼬노미야끼','랍스터','숯불닭갈비',
             '탄탄면','밀푀유나베','술찜','통찜','묵은지찜','김치찜','뽈찜','곱도리','쉬림프','씨푸드',
             '뇨끼','폭립','소꼬리찜','마라샹궈','동남아','에그인헬','빠네','찜닭','해물찜','블랙라벨',
             '오코노미야끼', '막창','리조또','활전복','된장','자미곱','동남아식','더원푸드','맛도령',
             '순살', '쿠킹','한우','소곱창전골','육통령', '된장찌개','당감댁', '캠핑용', '호텔식', '간편',
             '파우즈','레이식당', '팔공산', '얼큰이','허챠밍','돼고비','투다리', '직화', '직화구이',
             '찹스테이크','황토집','솔리보', '정인푸드몰', '고기남자','마이야르','돈이','냄새없는','생생',
             '쿡솜씨', '아히요', '도리명가', '로미랜드', '소제이','하정강', '양푼이','신사강', '파피요트', 
             '등갈비찜', '서민갑부','오징어볶음','쭈꾸미','소갈비찜']
             
word_list = "타코야끼, 붕어빵, 먹쉬돈나, 샌드위치, 바삭, 앙트레, 알떡, 타코, 햄버거, 엽떡, 화명동, 컵케이크, 케이크, 키키쿠쿠, 피자, 쿠키, 크리스마스, 핑크퐁, 수제, 물회, 족발, 마켓컬리, 부방대마켓, 소고기죽, 갈비살, 닭날개밥, 파는곳, 돌솥밥, 가바쌀밥, 그래놀라, 코코아볼, 큰공기, 흑산동인, 유산슬, 나물, 굴라쉬, 문어밥, 삼계죽, 묵은지, 게딱지밥, 얼바인몰, 내장죽, 청기와타운, 치킨마요, 토비코, 바질, 통오징어, 삼겹덮밥, 수정과, 영양밥, 전투식량, 야끼소바, 햇반, 컵반, 리조토, 스시, 통단팥죽, 야채죽, 삼시먹끼, 쇠고기죽, 그랑씨엘, 마구푸드, 새알팥죽, 하이밀마켓, 꼬마김밥, 솥밥, 전복밥, 기장끝집, 보일링씨푸드, 트러플, 굴대창전골, 간편하게, 즐기는, 혼밥, 맛있는, 주왕산가든, 승우아빠, 국내산, 엄마손, 곱도리, 팔당, 오징어, 피셔맨즈, 알찜, 터알, 키트키, 짜글이, 엄마가, 부찌의, 꽃게탕, 미가, 비건, 쵸이닷, 매드포갈릭, 알리오올리오, 엘본, 트러플페퍼, 쯔란, 마이셰프, 닭사메무쵸, 만득이, 부라더, 화이타, 유산슬, 바다해, 준식이, 매코마니, 흑돼지, 옥만이네, 눈꽃, 국물, 미미네, 케이터링, 청어람, 채선당, 말고, 키탤리, 칼국수, 편스토랑, 맹버칼, 봉골레, 간편식, 김추일, 숭의가든, 뽈뽀, 돈마호크, 류몽민, 서가앤쿡, 심쿡, 콩피, 스테이크, 깐쇼새우, 꼼장어, 박휘순대볶음, 진또배기, 랍스타, 쩐데이, 다올바리, 복선당, 홍은이, 은영이, 동태탕, 염소탕, 은주정, 택배, 햄찌와베찌, 수비드, 박가미, 돈까스, 똥반오반, 조가네, 하린이네, 우삼겹, 곰표, 매우매오, 안동인, 전동집, 쪽갈비, 어부의, 감동식당, 병철이네, 맛있지, 옥만이네, 흑태찜, 버거형, 상국이네, 로제, 아펠가모, 앳홈, 닥치고, 편스토랑, 월남쌈, 둘이서푸드, 고양옥, 라라관, 최저가, 포방터, 낙솝새, 호랑이굴, 저렴한, 마곱간, 은이세끼, 퀘사디아, 살치살, 널위한, 안지랑, 곱소반, 팔당, 하코야, 콩불, 심플리쿡, 쪽갈비구이, 부르기뇽, 셰일링, 보일링, 찜하고탕, 부찌의, 한탄강, 조방앞, 한마리, 오뎅탕, 샤브, 대가방, 똠양꿍, 태국요리, 고수, 느타리, 숙주, 미포함, 볶음면, 탐나는, 수비드, 양갈비, 도도나, 규카츠, 서프앤터프, 오근내, 말순씨, 골사장, 느루집, 연이네, 요리다움, 보일드, 등갈비, 바른씨, 알곤이찜, 슈바인학센, 파파쿡, 만들기, 순대국, 흑염소탕, 스끼야끼, 청담동, 순도리, 한구, 만두, 엘림, 들깨, 르포트, 미싀고, 박막례, 트레이더스, 맛좀봐라, 일산, 우육면, 아내의, 시그니처, 블랙킹, 타코야끼, 가져가요, 분모자, 거너실, 랜선, 오징어볶음, 쭈꾸미, 블랙라벨, 숯불, 닭갈비, 피코크, 곱도리탕, 오꼬노미야끼, 불 닭갈비, 밀키트, 낙곱새, 해신탕, 감바스, 자미곱, 마라탕, 동남아식, 탄탄면, 더원푸드, 프레시지, 빠에야, 맛도령, 순살, 쿠킹, 2인, 3인, 4인분, 리조또, 한우, 소곱창전골, 된장, 활전복, 육통령, 된장찌개, 당감댁, 캠핑용, 뇨끼, 호텔식, 간편, 리조또, 파우즈, 투움바, 레이식당, 팔공산, 얼큰이, 허챠밍, 에그인헬, 빠네, 돼고비, 투다리, 직화, 직화구이, 불닭갈비, 찹스테이크, 황토집, 가니쉬, 솔리보, 정인푸드몰, 고기남자, 폭립, 마이야르, 소곱창, 돈이, 냄새없는, 생생, 오코노미야끼, 마라샹궈, 쿡솜씨, 아히요, 도리명가, 찜닭, 우족찜, 술찜, 소갈비찜, 씨푸드, 쉬림프, 로미랜드, 김치찜, 소제이, 소꼬리찜, 해물찜, 하정강, 양푼이, 랍스터, 신사강, 파피요트, 등갈비찜, 서민갑부"
word_list = word_list.split(', ')

#230103 민규님이 추가한 txt파일 추가
%cd /content/drive/My Drive
with open('쿠팡재료띄어쓰기.txt', 'r') as f:
  lines = f.readlines()
  lines = str(lines).split(', ')
  for line in lines:
    word_list.append(line)

%cd /content/mecab-ko-dic-2.1.1-20180720/
file_data = get_nnpFile(word_list)
with open("./user-dic/nnp.csv", 'w', encoding='utf-8') as f:
  for line in file_data:
    f.write(line)

/content/mecab-ko-dic-2.1.1-20180720
/content/drive/My Drive
/content/mecab-ko-dic-2.1.1-20180720


In [40]:
len(word_list)

540

In [41]:
#설정 저장
!bash ./autogen.sh
!make
!sudo make install
!bash ./tools/add-userdic.sh

Looking in current directory for macros.
configure.ac:2: warning: AM_INIT_AUTOMAKE: two- and three-arguments forms are deprecated.  For more info, see:
configure.ac:2: http://www.gnu.org/software/automake/manual/automake.html#Modernize-AM_005fINIT_005fAUTOMAKE-invocation
 cd . && /bin/bash ./config.status Makefile 
config.status: creating Makefile
make: Nothing to be done for 'all'.
make[1]: Entering directory '/content/mecab-ko-dic-2.1.1-20180720'
make[1]: Nothing to be done for 'install-exec-am'.
 /bin/mkdir -p '/usr/local/lib/mecab/dic/mecab-ko-dic'
 /usr/bin/install -c -m 644 model.bin matrix.bin char.bin sys.dic unk.dic left-id.def right-id.def rewrite.def pos-id.def dicrc '/usr/local/lib/mecab/dic/mecab-ko-dic'
make[1]: Leaving directory '/content/mecab-ko-dic-2.1.1-20180720'
generating userdic...
nnp.csv
/content/mecab-ko-dic-2.1.1-20180720/tools/../model.def is not a binary model. reopen it as text mode...
reading /content/mecab-ko-dic-2.1.1-20180720/tools/../user-dic/nnp.csv .

# user nnp 사전에서도 우선순위 적용

In [42]:
nnp = pd.read_csv('user-nnp.csv', index_col = 0)
nnp['3821'] = 0
nnp = nnp.rename(columns={'3821':0})
nnp.to_csv('user-nnp.csv')

* make clean <br>
= 이전 컴파일 내용을 삭제
* make distclean<br>
 = 파일을 받고 압축을 푼 처음상태로 만드는것
* make install<br>
 = make를 통해 만들어진 설치파일(setup)을 설치를 하는 과정이다. 한마디로 build된 프로그램을 실행 할 수 있게 파일들을 알맞은 위치에다가 복사를 한다.

In [43]:
!make clean    #이전 컴파일 내용을 삭제

test -z "model.bin matrix.bin char.bin sys.dic unk.dic" || rm -f model.bin matrix.bin char.bin sys.dic unk.dic


In [44]:
!make install  #만들어진 파일 설치

/usr/local/libexec/mecab/mecab-dict-index -d . -o . -f UTF-8 -t UTF-8
reading ./unk.def ... 13
emitting double-array: 100% |###########################################| 
reading ./Place-station.csv ... 1145
reading ./VV.csv ... 7331
reading ./NNG.csv ... 208524
reading ./XSN.csv ... 124
reading ./EF.csv ... 1820
reading ./user-person.csv ... 1
reading ./EP.csv ... 51
reading ./NP.csv ... 342
reading ./XR.csv ... 3637
reading ./IC.csv ... 1305
reading ./CoinedWord.csv ... 148
reading ./Wikipedia.csv ... 36762
reading ./Hanja.csv ... 125750
reading ./NNP.csv ... 2371
reading ./ETN.csv ... 14
reading ./NNB.csv ... 140
reading ./Foreign.csv ... 11690
reading ./VX.csv ... 125
reading ./XPN.csv ... 83
reading ./XSV.csv ... 23
reading ./ETM.csv ... 133
reading ./Place.csv ... 30303
reading ./NR.csv ... 482
reading ./Inflect.csv ... 44820
reading ./NNBC.csv ... 677
reading ./user-place.csv ... 2
reading ./XSA.csv ... 19
reading ./MM.csv ... 453
reading ./Preanalysis.csv ... 5
reading ./Place-a

# 네이버데이터랩 인기검색어 파일에 적용

In [ ]:
%cd /content/drive/My Drive

naver_df = trend_df.copy()

In [ ]:
from konlpy.tag import Mecab

mecab = Mecab()
for column in range(6):
  naver_df.iloc[:, column] = naver_df.iloc[:, column].apply(lambda trend : " ".join(mecab.morphs(trend)))

## 네이버 가중치 추가

In [ ]:
naver_len = len(naver_df)
naver_df['가중치'] = [naver_weight/naver_len for naver_weight in range(naver_len,0, -1)]
naver_df.to_csv(str(year) + '-' + str(month) + '-' + str(day) + 'edited_navertrend.csv')
naver_df

In [ ]:
# 네이버트렌드 카테고리 : 상세카테고리
dict1 = {'밥/죽' : ['리조또', '한식'],
         '찌개/국' : ['전골', '생선탕', '찌개/짜글이', '알탕', '해물탕', '국', '마라탕'],
         '면/파스타' : ['면', '국수', '파스타'],
         '구이' : ['구이', '스테이크'],
         '볶음/튀김' : ['볶음', '감바스', '순대'],
         '조림/찜' : ['찜', '마파두부'],
         '간식/디저트' : ['샌드위치', '족발'],
         '전체' : ['씨푸드', '닭', '떡볶이', '무침', '쌈', '카레']}

# 쿠팡 상품명 띄어쓰기 적용

In [45]:
from konlpy.tag import Mecab
%cd /content/drive/My Drive

mecab = Mecab()
coupang = pd.read_csv('coupang_filtering.csv')
coupang['상품_전처리'] = coupang.상품.apply(lambda trend : " ".join(mecab.morphs(trend)))
coupang.to_csv('coupang_filtering_pre.csv')
coupang.head()

/content/drive/My Drive


,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,상품_전처리
0,0,1,1866720935,3172927841,71160443802,프레시지 대한곱창 곱창전골,"프레시지 대한곱창 곱창전골, 1594g, 1개",26270,1%,25900,"(100g당 1,625원)",4.5,11438,0,"양념육(소곱창, 소대창), 전골용 육수, 새송이버섯, 느타리버섯, 소고기, 두부, ...","양념육,소곱창,소대창,전골용 육수,새송이버섯,느타리버섯,소고기,두부,배추,양파,깻잎...",전골,프레시지 대한곱창 곱창전골
1,1,1,293421715,926408953,5302203368,마이셰프 밀푀유나베 & 칼국수,"마이셰프 밀푀유나베 & 칼국수, 1129g, 1개",16900,0,16900,"(100g당 1,497원)",4.5,49861,0,"소고기, 만두, 생면, 소스 2종, 육수, 표고버섯, 팽이버섯, 채소(배추, 청경채...","소고기,만두,생면,육수,표고버섯,팽이버섯,채소,배추,청경채,깻잎",전골,마이셰프 밀푀유 나베 & 칼국수
2,2,1,4926044090,6464437521,73758857275,곰곰 밀푀유 나베,"곰곰 밀푀유 나베, 1.2kg, 1세트",14990,0,14990,"(100g당 1,249원)",4.5,15915,0,"쇠고기, 새송이버섯, 표고버섯, 초간장소스, 칠리소스, 샤브용육수, 생면, 칼국수면...","쇠고기,새송이버섯,표고버섯,초간장소스,칠리소스,샤브용육수,생면,칼국수면,숙면,감자수...",전골,곰곰 밀푀유 나베
3,3,1,2202010823,3744622966,71729775756,미소프레쉬 6컵 푸짐한 대구탕,"미소프레쉬 6컵 푸짐한 대구탕, 1225g, 1개",14900,0,14900,"(100g당 1,216원)",4.5,12229,0,"해물, 채소, 연두부, 다시팩, 소스로 구성되어 있습니다.","해물,채소,연두부,다시팩,소스",생선탕,미소프레쉬 6컵 푸짐한 대구탕
4,4,1,1717552921,2923167957,70911802261,프레시지 더큰 햄가득 부대전골,"프레시지 더큰 햄가득 부대전골, 868g, 1개",14900,0,14900,"(100g당 1,717원)",5.0,37028,0,"소시지, 오뗄팜S, 프레스햄, 소고기, 치즈, 라면사리, 채소, 김치, 소스로 구성...","소시지,오뗄팜S,프레스햄,소고기,치즈,라면사리,채소,김치,소스",전골,프레시지 더큰 햄 가득 부대전골


In [48]:
mecab.pos('야채말이')

[('야채말이', 'NNP')]

In [50]:
coupang[coupang.상품.str.contains('굴부추')]

,Unnamed: 0,카테고리명,상품id,data-item-id,data-vendor-item-id,상품,상품명,정가,할인율,판매가,100g당_가격,별점,리뷰수,품절여부,구성정보,구성정보_전처리,상세카테고리,상품_전처리
284,290,7,6908772385,16642375559,83826520841,바다자리 생굴 해물 굴부추전 씨키트 2~3인분 320g + 야채 + 부침가루 세트,"바다자리 생굴 해물 굴부추전 씨키트 2~3인분 320g + 야채 + 부침가루 세트,...",19900,16%,16600,NaN,4.5,26,1,"굴, 오징어, 새우살, 쌀가루, 혼합 부침가루, 야채 세트(고추, 부추, 양파)로 ...","굴,오징어,새우살,쌀가루,고추,부추,양파",씨푸드,바다자리 생굴 해물 굴부추전 씨키트 2~3인분 320 g + 야채 + 부침 가루 세트


# 인기 검색어 띄어쓰기 전처리 TEST
- konlpy
- https://heegyukim.medium.com/symspell%EC%9D%84-%EC%9D%B4%EC%9A%A9%ED%95%9C-%ED%95%9C%EA%B8%80-%EB%A7%9E%EC%B6%A4%EB%B2%95-%EA%B5%90%EC%A0%95-2-%EB%B3%B5%ED%95%A9%EC%96%B4%EC%99%80-%EB%9D%84%EC%96%B4%EC%93%B0%EA%B8%B0-%EA%B5%90%EC%A0%95-7605ec2fcebc
- soynlp : https://github.com/lovit/soynlp

In [ ]:
from konlpy.tag import Hannanum, Kkma, Komoran, Okt
hannanum = Hannanum()
kkma = Kkma()
komoran = Komoran()
okt = Okt()

In [ ]:
word = '프리미엄부대찌개밀키트김치2인3인캠핑요리쿠킹박스술안주'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
word = '부산맛집밀키트(1회용냄비선물)당감댁곱도리탕닭볶음탕닭도리탕곱창안주캠핑용'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
word = '더원푸드시래기곱창전골밀키트'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
word = '일품정한우간편안주매콤한우우족찜밀키트650g'
print('hannanum : ', hannanum.morphs(word))
print('kkma : ', kkma.morphs(word))
print('komoran : ', komoran.morphs(word))
print('okt : ', okt.morphs(word))
print('mecab : ', mecab.morphs(word))

In [ ]:
# !pip3 install soynlp
# !wget https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt -O 2016-10-20.txt

In [ ]:
# ### 말 뭉치 생성
# from soynlp import DoublespaceLineCorpus

# # 문서 단위 말뭉치 생성 
# corpus = DoublespaceLineCorpus("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt")
# len(corpus)  # 문서의 갯수

In [ ]:
# import sklearn
# from soynlp.utils import DoublespaceLineCorpus
# from soynlp.noun import LRNounExtractor_v2
# from soynlp import DoublespaceLineCorpus

# corpus_path = '2016-10-20-news'
# sents = DoublespaceLineCorpus(corpus_path, iter_sent=True)

# noun_extractor = LRNounExtractor_v2(verbose=True)
# nouns = noun_extractor.train_extract(sents)
